# Part 1

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import time

--- Day 8: Seven Segment Search ---
You barely reach the safety of the cave when the whale smashes into the cave mouth, collapsing it. Sensors indicate another exit to this cave at a much greater depth, so you have no choice but to press on.

As your submarine slowly makes its way through the cave system, you notice that the four-digit seven-segment displays in your submarine are malfunctioning; they must have been damaged during the escape. You'll be in a lot of trouble without them, so you'd better figure out what's wrong.

Each digit of a seven-segment display is rendered by turning on or off any of seven segments named a through g:
```
  0:      1:      2:      3:      4:
 aaaa    ....    aaaa    aaaa    ....
b    c  .    c  .    c  .    c  b    c
b    c  .    c  .    c  .    c  b    c
 ....    ....    dddd    dddd    dddd
e    f  .    f  e    .  .    f  .    f
e    f  .    f  e    .  .    f  .    f
 gggg    ....    gggg    gggg    ....

  5:      6:      7:      8:      9:
 aaaa    aaaa    aaaa    aaaa    aaaa
b    .  b    .  .    c  b    c  b    c
b    .  b    .  .    c  b    c  b    c
 dddd    dddd    ....    dddd    dddd
.    f  e    f  .    f  e    f  .    f
.    f  e    f  .    f  e    f  .    f
 gggg    gggg    ....    gggg    gggg
 ```
So, to render a 1, only segments c and f would be turned on; the rest would be off. To render a 7, only segments a, c, and f would be turned on.

The problem is that the signals which control the segments have been mixed up on each display. The submarine is still trying to display numbers by producing output on signal wires a through g, but those wires are connected to segments randomly. Worse, the wire/segment connections are mixed up separately for each four-digit display! (All of the digits within a display use the same connections, though.)

So, you might know that only signal wires b and g are turned on, but that doesn't mean segments b and g are turned on: the only digit that uses two segments is 1, so it must mean segments c and f are meant to be on. With just that information, you still can't tell which wire (b/g) goes to which segment (c/f). For that, you'll need to collect more information.

For each display, you watch the changing signals for a while, make a note of all ten unique signal patterns you see, and then write down a single four digit output value (your puzzle input). Using the signal patterns, you should be able to work out which pattern corresponds to which digit.

For example, here is what you might see in a single entry in your notes:
```
acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab |
cdfeb fcadb cdfeb cdbaf
```
(The entry is wrapped here to two lines so it fits; in your notes, it will all be on a single line.)

Each entry consists of ten unique signal patterns, a | delimiter, and finally the four digit output value. Within an entry, the same wire/segment connections are used (but you don't know what the connections actually are). The unique signal patterns correspond to the ten different ways the submarine tries to render a digit using the current wire/segment connections. Because 7 is the only digit that uses three segments, dab in the above example means that to render a 7, signal lines d, a, and b are on. Because 4 is the only digit that uses four segments, eafb means that to render a 4, signal lines e, a, f, and b are on.

Using this information, you should be able to work out which combination of signal wires corresponds to each of the ten digits. Then, you can decode the four digit output value. Unfortunately, in the above example, all of the digits in the output value (cdfeb fcadb cdfeb cdbaf) use five segments and are more difficult to deduce.

For now, focus on the easy digits. Consider this larger example:
```
be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb |
fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec |
fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef |
cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega |
efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga |
gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf |
gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf |
cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd |
ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg |
gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc |
fgae cfgab fg bagce
```
Because the digits 1, 4, 7, and 8 each use a unique number of segments, you should be able to tell which combinations of signals correspond to those digits. Counting only digits in the output values (the part after | on each line), in the above example, there are 26 instances of digits that use a unique number of segments (highlighted above).

In the output values, how many times do digits 1, 4, 7, or 8 appear?

In [2]:
df = pd.read_csv('dec8_input.csv')
df.head()

,input,delimiter,output
0,gbcefa eac acfbg ae dcabfg begcdaf ecgba fgaed...,|,cbgfa gedcb fgecab fbagdc
1,fb adceb febagd edgafcb cbfega gecadf dgfb dfe...,|,gbdf gbfd fgebac dfega
2,dgabec cfgeb cfb cefd cdabfg gbdce fc begdfc g...,|,bcfdaeg cf fc gecabdf
3,gadfce acdgf bgaec gbfdca acgbd dceagfb agfdbe...,|,dgb cbadg cagdb cfbd
4,edcbafg ab ecfdb dacfeg bcag aeb fgaebd bfecga...,|,ba eab cdgefa gcab


In [3]:
df_test = pd.read_csv('dec8_test.csv')
df_test

,input,delimiter,output
0,aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb d...,|,gecf egdcabf bgf bfgea
1,bdfegc cbegaf gecbf dfcage bdacg ed bedf ced a...,|,ed bcgafe cdgba cbgef
2,be cfbegad cbdgef fgaecd cgeb fdcge agebfd fec...,|,fdgacbe cefdb cefbgd gcbe
3,dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dac...,|,cefg dcbef fcge gbcadfe
4,edbfga begcd cbg gc gcadebf fbgde acbgfd abcde...,|,fcgedb cgb dgebacf gc
5,egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab...,|,gbdfcae bgc cg cgb
6,fbegcd cbd adcefb dageb afcb bc aefdc ecdab fg...,|,efabcd cedba gadfec cb
7,fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac...,|,cg cg fdcagb cbg
8,fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfad...,|,gebdcfa ecba ca fadegcb
9,gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge ...,|,fgae cfgab fg bagce


In [7]:
def count_per_entry(output):
    
    target_lens = [2,4,3,7] 
    
    outputs = output.split(" ")
    lens = [len(x) for x in outputs]
    
    lens_of_interest = [x for x in lens if x in target_lens]
    
    outputs_of_interest = [map_len_to_output[x] for x in lens_of_interest]
    
    num_counts.update(outputs_of_interest)

In [8]:
map_len_to_output =  {
    2:1,
    4:4,
    3:7,
    7:8
}

In [9]:
# 1 -- 2, 4 -- 4, 7 -- 3, 8 -- 7

In [10]:
num_counts = Counter()

df['output'].apply(lambda x: count_per_entry(x))

0      None
1      None
2      None
3      None
4      None
       ... 
195    None
196    None
197    None
198    None
199    None
Name: output, Length: 200, dtype: object

In [11]:
sum(num_counts.values())

412

# part 2

--- Part Two ---
Through a little deduction, you should now be able to determine the remaining digits. Consider again the first example above:

acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab |
cdfeb fcadb cdfeb cdbaf
After some careful analysis, the mapping between signal wires and segments only make sense in the following configuration:
```
 dddd
e    a
e    a
 ffff
g    b
g    b
 cccc
 ```
So, the unique signal patterns would correspond to the following digits:
```
acedgfb: 8
cdfbe: 5
gcdfa: 2
fbcad: 3
dab: 7
cefabd: 9
cdfgeb: 6
eafb: 4
cagedb: 0
ab: 1
```
Then, the four digits of the output value can be decoded:
```
cdfeb: 5
fcadb: 3
cdfeb: 5
cdbaf: 3
```
Therefore, the output value for this entry is 5353.

Following this same process for each entry in the second, larger example above, the output value of each entry can be determined:
```
fdgacbe cefdb cefbgd gcbe: 8394
fcgedb cgb dgebacf gc: 9781
cg cg fdcagb cbg: 1197
efabcd cedba gadfec cb: 9361
gecf egdcabf bgf bfgea: 4873
gebdcfa ecba ca fadegcb: 8418
cefg dcbef fcge gbcadfe: 4548
ed bcgafe cdgba cbgef: 1625
gbdfcae bgc cg cgb: 8717
fgae cfgab fg bagce: 4315
```
Adding all of the output values in this larger example produces 61229.

For each entry, determine all of the wire/segment connections and decode the four-digit output values. What do you get if you add up all of the output values?

In [12]:
def sort_string(string):
    sorted_characters = sorted(string)
    string = "".join(sorted_characters)
    return string

In [13]:
def deduce(signal):
    
    def find_full_subset(string,substring, length):
        if len(string) == length:
            return (all(x in list(string) for x in list(substring)))
    
    signals = signal.split(" ")
    signal_mapping = {}
    
    # Find 1, 4, 7, 8
    
    # 1 is length 2
    one = [x for x in signals if len(x) == 2][0]
    signal_mapping[sort_string(one)] = 1
    signals.remove(one)

    # 4 is length 4
    four = [x for x in signals if len(x) == 4][0]
    signal_mapping[sort_string(four)] = 4
    signals.remove(four)
    
    # 7 is length 3
    seven = [x for x in signals if len(x) == 3][0]
    signal_mapping[sort_string(seven)] = 7
    signals.remove(seven)
    
    # 8 is length 7
    eight = [x for x in signals if len(x) == 7][0]
    signal_mapping[sort_string(eight)] = 8
    signals.remove(eight)

    # 3 is (only) five-digit one that fully contains 7
    three = [x for x in signals if find_full_subset(x,seven,5)][0]
    signal_mapping[sort_string(three)] = 3
    signals.remove(three)   
    
    # 9 is (only) six-digit one that fully contains 3
    nine = [x for x in signals if find_full_subset(x,three,6)][0]
    signal_mapping[sort_string(nine)] = 9
    signals.remove(nine)       
    
    # of remaining 5s and 6s
    # Can identify 5 and 6 because 6 fully contains 5
    len_five = [x for x in signals if len(x) == 5]
    len_six = [x for x in signals if len(x) == 6]
    
    for fives in len_five:
        for sixes in len_six:
            if find_full_subset(sixes,fives,6):
                five = fives
                signal_mapping[sort_string(five)] = 5
                signals.remove(five)
                
                six = sixes
                signal_mapping[sort_string(six)] = 6
                signals.remove(six)
                
    
    # Remaining 5 is 2
    two = [x for x in signals if len(x) == 5][0]
    signal_mapping[sort_string(two)] = 2
    signals.remove(two)
    
    # Remaining 6 is 0
    zero = signals[0]
    signal_mapping[sort_string(zero)] = 0
    signals.remove(zero)
  
    return signal_mapping

In [188]:
def apply_mapping(mapping,output):
    
    output_value = ''
    
    outputs = output.split(" ")
    
    # For each output, get the value
    for output in outputs:
        output_value = output_value + str(mapping[sort_string(output)])
    # concatenate
    return int(output_value)

In [190]:
def deduce_and_map(row):
    
    mapping = deduce(row.input)
    
    output_value = apply_mapping(mapping, row.output)
    
    return output_value

In [202]:
total_count = 0

for i,row in df.iterrows():
    total_count += deduce_and_map(row)
    
print(total_count)

978171
